# 4 Qubit Chip

##### Run Me :D  

Start Qiskit Metal GUI

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

design = designs.DesignPlanar()
gui = MetalGUI(design)


This code will make chip for you! It might take up to a minute! 

In [1]:

# if you disable the next line, then you will need to delete a component [<component>.delete()] before recreating it
design.overwrite_enabled = True

from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.components.interconnects.meandered import RouteMeander
from qiskit_metal.components.interconnects.anchored_path import RouteAnchors
from qiskit_metal.components.interconnects.pathfinder import RoutePathfinder
from qiskit_metal.components.connectors.open_to_ground import OpenToGround
from qiskit_metal.components.passives.launchpad_wb import LaunchpadWirebond
from qiskit_metal.components.passives.launchpad_wb_coupled import LaunchpadWirebondCoupled



design.variables['cpw_width'] = '10 um'
design.variables['cpw_gap'] = '6 um'

TransmonPocket.get_template_options(design)
transmon_options = dict(
    connection_pads=dict(
        a = dict(loc_W=+1, loc_H=-1, pad_width='70um', cpw_extend = '50um'), 
        b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um'),
        c = dict(loc_W=-1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
    ),
    gds_cell_name='FakeJunction_01'
)

offset_tm = 69  #we the transmon slightly out of center-line

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2420um', pos_y=f'{offset_tm}um', **transmon_options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='0um', pos_y='-857.6um', orientation = '270', **transmon_options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2420um', pos_y=f'{offset_tm}um', orientation = '180', **transmon_options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='0um', pos_y='+857.6um', orientation = '90', **transmon_options))

gui.rebuild()
gui.autoscale()



RouteMeander.get_template_options(design)
fillet='100um'
cpw_options = Dict(
    lead=Dict(
        start_straight='100um',
        end_straight='250um'),
    fillet=fillet
    )


def connect(cpw_name: str, pin1_comp_name: str, pin1_comp_pin: str, pin2_comp_name: str, pin2_comp_pin: str,
            length: str, asymmetry='0 um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(
                component=pin1_comp_name,
                pin=pin1_comp_pin),
            end_pin=Dict(
                component=pin2_comp_name,
                pin=pin2_comp_pin)),
        total_length=length)
    myoptions.update(cpw_options)
    myoptions.meander.asymmetry = asymmetry
    return RouteMeander(design, cpw_name, myoptions)

asym = 500
cpw1 = connect('cpw1', 'Q1', 'c', 'Q4', 'b', '9500um', f'-{asym-1.25*offset_tm}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q4', 'c', '9500um', f'+{asym-1.25*offset_tm}um')
cpw3 = connect('cpw3', 'Q3', 'c', 'Q2', 'b', '9500um', f'-{asym+0.75*offset_tm}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q2', 'c', '9500um', f'+{asym+0.75*offset_tm}um')

gui.rebuild()
gui.autoscale()

# V1 - Corners
p1_c = LaunchpadWirebond(design, 'P1_C', options = dict(pos_x='3545um', pos_y='2812um', orientation='270', lead_length='0um'))
p2_c = LaunchpadWirebond(design, 'P2_C', options = dict(pos_x='3545um', pos_y='-2812um', orientation='90', lead_length='0um'))
p3_c = LaunchpadWirebond(design, 'P3_C', options = dict(pos_x='-3545um', pos_y='-2812um', orientation='90', lead_length='0um'))
p4_c = LaunchpadWirebond(design, 'P4_C', options = dict(pos_x='-3545um', pos_y='2812um', orientation='270', lead_length='0um'))

# V2
p1_q = LaunchpadWirebondCoupled(design, 'P1_Q', options = dict(pos_x='4020um', pos_y='0', orientation='180', lead_length='30um'))
p2_q = LaunchpadWirebondCoupled(design, 'P2_Q', options = dict(pos_x='-990um', pos_y='-2812um', orientation='90', lead_length='30um'))
p3_q = LaunchpadWirebondCoupled(design, 'P3_Q', options = dict(pos_x='-4020um', pos_y='0', orientation='0', lead_length='30um'))
p4_q = LaunchpadWirebondCoupled(design, 'P4_Q', options = dict(pos_x='990um', pos_y='2812um', orientation='270', lead_length='30um'))

gui.rebuild()
gui.autoscale()



asym = 150
cpw_options = Dict(
    lead=Dict(
        start_straight='430um',
        end_straight='0um'),
    fillet=fillet
    )
ol1 = connect('ol1', 'Q1', 'a', 'P1_Q', 'tie', '8.6 mm', f'+{asym}um')
ol3 = connect('ol3', 'Q3', 'a', 'P3_Q', 'tie', '8.6 mm', f'+{asym}um')

asym = 200
cpw_options = Dict(
    lead=Dict(
        start_straight='535um',
        end_straight='0um'),
    fillet=fillet
    )
ol2 = connect('ol2', 'Q2', 'a', 'P2_Q', 'tie', '8.6 mm', f'+{asym}um')
ol4 = connect('ol4', 'Q4', 'a', 'P4_Q', 'tie', '8.6 mm', f'+{asym}um')

gui.rebuild()
gui.autoscale()

offset_gp = 21
otg1 = OpenToGround(design, 'OTG1', options = dict(pos_x='2780um', pos_y=f'{offset_tm+offset_gp}um', rotation='270', gap='cpw_gap', width='cpw_width'))
otg2 = OpenToGround(design, 'OTG2', options = dict(pos_x=f'{offset_gp}um', pos_y='-1218um', rotation='180', gap='cpw_gap', width='cpw_width'))
otg3 = OpenToGround(design, 'OTG3', options = dict(pos_x='-2780um', pos_y=f'{offset_tm-offset_gp}um', rotation='90', gap='cpw_gap', width='cpw_width'))
otg4 = OpenToGround(design, 'OTG4', options = dict(pos_x=f'{-offset_gp}um', pos_y='1218um', rotation='0', gap='cpw_gap', width='cpw_width'))

gui.rebuild()
gui.autoscale()


from collections import OrderedDict
jogsA_in = OrderedDict()
jogsA_in[0] = ["R", '200um']
jogsA_in[1] = ["L", '1800um']

options_line_cl1 = {'pin_inputs': 
            {'start_pin': {'component': 'OTG1', 'pin': 'open'}, 
             'end_pin': {'component': 'P1_C', 'pin': 'tie'}},
            'lead': {'start_straight': '20um', 'end_straight': '225um', 'start_jogged_extension': jogsA_in},
            'fillet': fillet
            }
cl1 = RouteAnchors(design, 'line_cl1', options_line_cl1)

options_line_cl3 = {'pin_inputs': 
            {'start_pin': {'component': 'OTG3', 'pin': 'open'}, 
             'end_pin': {'component': 'P3_C', 'pin': 'tie'}},
            'lead': {'start_straight': '20um', 'end_straight': '225um', 'start_jogged_extension': jogsA_in},
            'fillet': fillet
            }
cl3 = RouteAnchors(design, 'line_cl3', options_line_cl3)

gui.rebuild()
gui.autoscale()

jogsB_in = OrderedDict()
jogsB_in[0] = ["R", '300um']
jogsB_in[1] = ["L", '1200um']

anchors2c = OrderedDict()
anchors2c[0] = np.array([2, -2.5])

options_line_cl2 = {'pin_inputs': 
            {'start_pin': {'component': 'OTG2', 'pin': 'open'}, 
             'end_pin': {'component': 'P2_C', 'pin': 'tie'}},
            'lead': {'start_straight': '20um', 'end_straight': '225um',
                     'start_jogged_extension': jogsB_in},
            'anchors': anchors2c,
            'fillet': fillet
            }

cl2 = RouteAnchors(design, 'line_cl2', options_line_cl2)

anchors4c = OrderedDict()
anchors4c[0] = np.array([-2, 2.5])

options_line_cl4 = {'pin_inputs': 
            {'start_pin': {'component': 'OTG4', 'pin': 'open'}, 
             'end_pin': {'component': 'P4_C', 'pin': 'tie'}},
            'lead': {'start_straight': '20um', 'end_straight': '225um',
                     'start_jogged_extension': jogsB_in},
            'anchors': anchors4c,
            'fillet': fillet
            }

cl4 = RouteAnchors(design, 'line_cl4', options_line_cl4)

gui.rebuild()
gui.autoscale()

04:47PM 53s WARNING [check_lengths]: For path table, component=line_cl1, key=trace has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapley geometry.
04:47PM 53s WARNING [check_lengths]: For path table, component=line_cl1, key=cut has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapley geometry.
04:47PM 53s WARNING [check_lengths]: For path table, component=line_cl3, key=trace has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapley geometry.
04:47PM 53s WARNING [check_lengths]: For path table, component=line_cl3, key=cut has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapley geometry.
04:47PM 54s WARNING [check_lengths]: For path table, component=line_cl1, key=trace has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapley geometry.
04:47PM 54s WARNING [check_lengths]: For path tab



You just used Metal to make a 4 qubit chip with each qubit having 2 couplings. It is a good test chip for experimental purposes (testing qubit gates, quality of the designs, etc.)



But you want to make _your own chip_ for your own experiments and labs

That's what the rest of our tutorials are for.

So go learn how to use Metal so you can create unique chips for your specific research.

We're so excited your joining us on the Metal journey.
Thank you~ and Good Luck~